In [1]:
%defaultDatasource jdbc:h2:mem:db

In [2]:
DROP TABLE IF EXISTS Pais;
CREATE TABLE Pais (
    nome VARCHAR(60),            -- eliminar ambiguidade!
    sigla VARCHAR(3) NOT NULL,
    regiao VARCHAR(30),
    classificacaoRenda VARCHAR(30),
    Gini INTEGER,
    PIBperCapita DOUBLE,
    IDH INTEGER,
    populacao INTEGER,
    PRIMARY KEY(sigla)
) AS SELECT
    country,
    label,
    region,
    wb_class,
    gini,
    gdbPPPperCapita,
    hdi,
    populationTotal
FROM CSVREAD('../../data/processed/countries.csv');

DROP TABLE IF EXISTS Policies;
CREATE TABLE Policies (
    id VARCHAR(4) NOT NULL,
    policy VARCHAR(150) NOT NULL,
    type VARCHAR(12),
    PRIMARY KEY(id)
) AS SELECT
    id,
    policy,
    type
FROM CSVREAD('../../data/processed/policies.csv');

DROP TABLE IF EXISTS CountryPolicies;
CREATE TABLE CountryPolicies (
    country VARCHAR(60) NOT NULL,
    policyId VARCHAR(4) NOT NULL,
    isNumeric BOOL NOT NULL,
    value VARCHAR(100),
    numericValue INTEGER,
    PRIMARY KEY(country, policyId)
) AS SELECT
    country ,
    policyId,
    isNumeric,
    value,
    numericValue
type
FROM CSVREAD('../../data/processed/country_policies.csv');

In [21]:
DROP VIEW IF EXISTS PoliticasRenda;
DROP VIEW IF EXISTS PoliticasPaises;
CREATE VIEW PoliticasPaises AS
SELECT Pais.nome nome, Pais.classificacaoRenda renda, COUNT (*) politicas
    FROM Pais JOIN CountryPolicies CP ON Pais.nome = CP.country
    WHERE CP.isNumeric = False AND
          CP.value <> 'No'
    GROUP BY Pais.nome;
    
CREATE VIEW PoliticasRenda AS
SELECT P1.renda, SUM(P1.politicas)/(SELECT COUNT(*) FROM PoliticasPaises P2 WHERE P2.politicas <> 0 AND P2.renda = p1.renda) Media_Politicas
    FROM PoliticasPaises P1
    WHERE P1.politicas <> 0
    GROUP BY P1.renda;
CALL CSVWRITE('../../saida/politicas-x-renda.csv', 'SELECT * FROM PoliticasRenda');
SELECT * FROM PoliticasRenda;